# Recommender ChatBot: Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

BASE_PATH    = '../..'
API_PATH     = f'{BASE_PATH}/chat-bot-api'
LIB_PATH     = f'{BASE_PATH}/lib'


os.environ['TMP_PATH']         = f'{BASE_PATH}/tmp'
os.environ['DATASET_PATH']     = f'{BASE_PATH}/datasets'
os.environ['WEIGHTS_PATH']     = f'{BASE_PATH}/weights'
os.environ['METRICS_PATH']     = f'{BASE_PATH}/metrics'

In [3]:
import sys
sys.path.append(LIB_PATH)
sys.path.append(API_PATH)

import pytorch_common.util as pu

import client

## Setup

In [4]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [5]:
api_client = client.RecChatBotV1ApiClient()

In [6]:
user_id = "adrianmarino@gmail.com"

In [7]:
api_client.remove_interactions_by_user_id(user_id)

2024-03-16 09:44:55,834 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/adrianmarino@gmail.com


In [8]:
api_client.interactions_by_user(user_id)

[]

In [9]:
%%time

query = {
    "message": {
        "author": "adrianmarino@gmail.com",
        "content": "I want see marvel movies"
    },
    "settings": {
        "llm"                                   : "llama2-7b-chat",
        "retry"                                 : 2,
        "plain"                                 : False,
        "include_metadata"                      : True,
        "rag": {
            "shuffle"                           : True,
            "candidates_limit"                  : 50,
            "llm_response_limit"                : 50,
            "recommendations_limit"             : 5,
            "similar_items_augmentation_limit"  : 5,
            "not_seen": True
        },
        "collaborative_filtering": {
            "shuffle"                           : False,
            "candidates_limit"                  : 50,
            "llm_response_limit"                : 50,
            "recommendations_limit"             : 5,
            "similar_items_augmentation_limit"  : 5,
            "text_query_limit"                  : 5000,
            "k_sim_users"                       : 10,
            "random_selection_items_by_user"    : 0.5,
            "max_items_by_user"                 : 10,
            "min_rating_by_user"                : 3.5,
            "not_seen"                          : True
        }
    }
}

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


In [10]:
result = api_client.recommendations(query)
result.items

2024-03-16 09:44:56,574 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/recommendations


[
 (
     Id         : 102125
     Title      : Iron Man 3
     Release    : 2013
     Genres     : ['action', 'sci-fi', 'thriller', 'imax']
     Rating     : 3.2736842105
     Poster     : http://image.tmdb.org/t/p/w500/qhPtAc1TKbMPqNvcdXSOn9Bn7hZ.jpg
     Description: Action-packed superhero movie with a great performance by robert downey jr.
 ),
 
 (
     Id         : 59315
     Title      : Iron Man
     Release    : 2013
     Genres     : ['action', 'adventure', 'sci-fi']
     Rating     : 3.8469387755
     Poster     : http://image.tmdb.org/t/p/w500/78lPtwv72eTNqFW9COBYI0dWDJa.jpg
     Description: Action-packed superhero movie with a great performance by robert downey jr.
 ),
 
 (
     Id         : 77561
     Title      : Iron Man 2
     Release    : 2013
     Genres     : ['action', 'adventure', 'sci-fi', 'thriller', 'imax']
     Rating     : 3.4307692308
     Poster     : http://image.tmdb.org/t/p/w500/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg
     Description: Action-packed superhero m

In [11]:
result.items[0].vote(5)
api_client.interactions_by_user(user_id)

2024-03-16 09:45:11,034 - INFO - GET http://nonosoft.ddns.net:8080/api/v1/interactions/make/adrianmarino@gmail.com/102125/5
2024-03-16 09:45:11,059 - INFO - 204


[{'user_id': 'adrianmarino@gmail.com',
  'item_id': '102125',
  'rating': 5.0,
  'timestamp': '2024-03-16 09:33:10'}]

In [12]:
result = api_client.recommendations(query)
result.items

2024-03-16 09:45:11,091 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/recommendations


[
 (
     Id         : 2571
     Title      : Matrix, The
     Release    : 1999
     Genres     : ['action', 'sci-fi', 'thriller']
     Rating     : 4.2724014337
     Poster     : http://image.tmdb.org/t/p/w500/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg
     Description: A classic science fiction movie with mind-bending special effects and a thought-provoking storyline.
 ),
 
 (
     Id         : 79132
     Title      : Inception
     Release    : 2010
     Genres     : ['action', 'crime', 'drama', 'mystery', 'sci-fi', 'thriller', 'imax']
     Rating     : 4.2242833052
     Poster     : http://image.tmdb.org/t/p/w500/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg
     Description: A mind-bending action movie that explores the concept of shared dreaming, featuring a talented cast led by leonardo dicaprio.
 ),
 
 (
     Id         : 109487
     Title      : Interstellar
     Release    : 2014
     Genres     : ['sci-fi', 'imax']
     Rating     : 4.0629067245
     Poster     : http://image.tmdb.org/t/p/w500/gEU2